In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm

from mpmath import besseljzero
from scipy.special import gamma
from scipy.special import jv, iv, ive
from scipy.optimize import differential_evolution
from scipy.interpolate import interp1d

from scipy.stats import pearsonr
from sklearn.metrics import r2_score

import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
def simulate_HSDM_3D(a, mu, ndt, sigma=1, dt=0.001):
    x = np.zeros(mu.shape)
    
    rt = 0
    
    while np.linalg.norm(x, 2) < a(rt):
        x += mu*dt + sigma*np.sqrt(dt)*np.random.normal(0, 1, mu.shape)
        rt += dt
    
    theta1 = np.arctan2(np.sqrt(x[2]**2 + x[1]**2), x[0])
    theta2 = np.arctan2(x[2], x[1])   
    
    return rt+ndt, (theta1, theta2)

In [3]:
def k(a, da, t, q, sigma=2):
    return 0.5 * (q - 0.5*sigma - da(t))

def psi(a, da, t, z, tau, q, sigma=2):
    kk = k(a, da, t, q, sigma)
    
    if 2*np.sqrt(a(t)*z)/(sigma*(t-tau))<=700:
        term1 = 1./(sigma*(t - tau)) * np.exp(- (a(t) + z)/(sigma*(t-tau)))
        term2 = (a(t)/z)**(0.5*(q-sigma)/sigma)
        term3 = da(t) - (a(t)/(t-tau)) + kk
        term4 = iv(q/sigma-1, 2*np.sqrt(a(t)*z)/(sigma*(t-tau)))
        term5 = (np.sqrt(a(t)*z)/(t-tau)) * iv(q/sigma, 2*np.sqrt(a(t)*z)/(sigma*(t-tau)))
    else:
        term1 = 1./(sigma*(t - tau))
        term2 = (a(t)/z)**(0.5*(q-sigma)/sigma)
        term3 = da(t) - (a(t)/(t-tau)) + kk
        term4 = ive(q/sigma-1, (a(t) + z)/(sigma*(t-tau)))
        term5 = (np.sqrt(a(t)*z)/(t-tau)) * ive(q/sigma, (a(t) + z)/(sigma*(t-tau)))
    
    return term1 * term2 * (term3 * term4 + term5)

def ie_bessel_fpt(a, da, q, z, sigma=2, dt=0.1, T_max=2):
    g = [0]
    T = [0]
    g.append(-2*psi(a, da, dt, z, 0, q, sigma))
    T.append(dt)
    
    for n in range(2, int(T_max/dt)+2):
        s = -2 * psi(a, da, n*dt, z, 0, q, sigma)

        for j in range(1, n):
            s += 2 * dt * g[j] * psi(a, da, n*dt, a(j*dt), j*dt, q, sigma)

        g.append(s)
        T.append(n*dt)
        
    g = np.asarray(g)
    T = np.asarray(T)
    
    gt = interp1d(T, g)
    return gt

In [4]:
def HSDM_3D_likelihood(prms, RT, Theta):
    a = lambda t: prms[0]**2
    da = lambda t: 0
    ndt = prms[1]
    mu = np.array([prms[2], prms[3], prms[4]])
    
    fpt = ie_bessel_fpt(a, da, mu.shape[0], 0.0001, 
                        dt=0.1, T_max=max(RT))
    log_lik = 0
    for i in range(len(RT)):
        rt, theta = RT[i], Theta[i]
        if rt - ndt > 0.001 and ndt >= 0:
            mu_dot_x0 = mu[0]*np.cos(theta[0])
            mu_dot_x1 = mu[1]*np.sin(theta[0])*np.cos(theta[1]) 
            mu_dot_x2 = mu[2]*np.sin(theta[0])*np.sin(theta[1])
            term1 = prms[0] * (mu_dot_x0 + mu_dot_x1 + mu_dot_x2)
            term2 = 0.5 * np.linalg.norm(mu, 2)**2 * (rt-ndt)
            
            density = np.exp(term1 - term2) * fpt(rt - ndt)
            
            if 0.1**14 < density:
                log_lik += -np.log(density)
            else:
                log_lik += -np.log(0.1**14)
        else:
            log_lik += -np.log(0.1**14)
        
    return log_lik

In [5]:
recovery_df = {'threshold_true': [],
               'threshold_estimate': [],
               'ndt_true': [],
               'ndt_estimate': [],
               'mu1_true': [],
               'mu1_estimate': [],
               'mu2_true': [],
               'mu2_estimate': [],
               'mu3_true': [],
               'mu3_estimate': []}

min_threshold = 0.5
max_threshold = 6

min_ndt = 0.1
max_ndt = 1

min_mu = -5
max_mu = 5

In [6]:
for n in tqdm(range(100)):
    threshold = np.random.uniform(min_threshold, max_threshold)
    a = lambda t: threshold
    ndt = np.random.uniform(min_ndt, max_ndt)
    mu = np.array([np.random.uniform(min_mu, max_mu), 
                   np.random.uniform(min_mu, max_mu),
                   np.random.uniform(min_mu, max_mu)])
    
    
    recovery_df['threshold_true'].append(threshold)
    recovery_df['ndt_true'].append(ndt)
    recovery_df['mu1_true'].append(mu[0])
    recovery_df['mu2_true'].append(mu[1])
    recovery_df['mu3_true'].append(mu[2])
    
    RT = []
    Theta = []
    
    for i in range(250):
        rt, theta = simulate_HSDM_3D(a, mu, ndt)
        RT.append(rt)
        Theta.append(theta)
    
    min_ans = differential_evolution(HSDM_3D_likelihood,
                                     args=(RT, Theta), 
                                     bounds=[(min_threshold, max_threshold), (min_ndt, max_ndt),
                                             (min_mu, max_mu), (min_mu, max_mu), (min_mu, max_mu)])
    
    recovery_df['threshold_estimate'].append(min_ans.x[0])
    recovery_df['ndt_estimate'].append(min_ans.x[1])
    recovery_df['mu1_estimate'].append(min_ans.x[2])
    recovery_df['mu2_estimate'].append(min_ans.x[3])
    recovery_df['mu3_estimate'].append(min_ans.x[4])
    
recovery_df = pd.DataFrame(recovery_df)

100%|█████████████████████████████████████████| 150/150 [31:35<00:00, 12.64s/it]


In [7]:
recovery_df

,threshold_true,threshold_estimate,ndt_true,ndt_estimate,mu1_true,mu1_estimate,mu2_true,mu2_estimate,mu3_true,mu3_estimate
0,2.588410,2.478106,0.666014,0.770566,-1.210742,-1.259883,1.451178,1.614294,1.000703,0.943332
1,2.537226,2.486873,0.896369,0.935545,2.863080,2.908826,-0.354840,-0.302329,-1.053448,-1.047487
2,3.753306,4.013130,0.183874,0.136078,0.318880,0.252841,0.615971,0.709169,-2.942493,-3.044346
3,3.512905,3.585713,0.359358,0.354727,-1.607270,-1.782679,2.244787,2.221546,-1.087949,-1.108516
4,3.667187,3.589208,0.264121,0.352721,-4.840621,-5.000000,-4.196422,-4.821662,-1.934171,-2.127014
...,...,...,...,...,...,...,...,...,...,...
145,5.178028,5.445376,0.258891,0.246199,-1.021972,-0.886323,3.230351,3.292467,1.076835,1.129457
146,2.826036,2.791857,0.469148,0.518681,-1.862365,-1.964134,-2.852495,-2.938478,-0.197802,-0.195747
147,1.386563,1.203943,0.430377,0.525166,-2.551996,-3.355279,-2.095074,-2.769515,3.899632,4.933001
148,2.923955,3.191889,0.756436,0.799368,2.495942,2.758271,-2.112419,-2.450526,2.481460,2.802795


In [8]:
recovery_df.corr()

,threshold_true,threshold_estimate,ndt_true,ndt_estimate,mu1_true,mu1_estimate,mu2_true,mu2_estimate,mu3_true,mu3_estimate
threshold_true,1.000000,0.985996,0.008930,0.088474,0.033004,0.040546,0.159710,0.169052,0.107402,0.101765
threshold_estimate,0.985996,1.000000,0.040598,0.098549,0.023529,0.033232,0.157074,0.166603,0.101903,0.094651
ndt_true,0.008930,0.040598,1.000000,0.976305,-0.102495,-0.108266,0.064827,0.067628,0.026521,0.019858
ndt_estimate,0.088474,0.098549,0.976305,1.000000,-0.059969,-0.067106,0.063288,0.066066,0.022502,0.016477
mu1_true,0.033004,0.023529,-0.102495,-0.059969,1.000000,0.992425,-0.003889,-0.005997,0.057898,0.060683
mu1_estimate,0.040546,0.033232,-0.108266,-0.067106,0.992425,1.000000,0.007868,0.004453,0.077290,0.081008
mu2_true,0.159710,0.157074,0.064827,0.063288,-0.003889,0.007868,1.000000,0.992995,0.091033,0.081318
mu2_estimate,0.169052,0.166603,0.067628,0.066066,-0.005997,0.004453,0.992995,1.000000,0.088216,0.078069
mu3_true,0.107402,0.101903,0.026521,0.022502,0.057898,0.077290,0.091033,0.088216,1.000000,0.994736
mu3_estimate,0.101765,0.094651,0.019858,0.016477,0.060683,0.081008,0.081318,0.078069,0.994736,1.000000


In [9]:
file_name = 'IE_3d_recovery_1.csv'
old_recovery_data = pd.read_csv(file_name, index_col=0)
recovery_df = pd.concat([old_recovery_data, 
                         recovery_df]).reset_index(drop=True)
recovery_df.to_csv(file_name)